In [1]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=78f6fda871cbf19fe32e1208450af64656278176dad8fc2fbe29677bb56efd80
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 880 kB 43.9 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 41.4 MB/s 
     |████████████████████████████████| 6.6 MB 31.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=dd28ea6ee84e8c11461a90c3f8d458c5d24190c452cb3a35fee19f6dbef491e3
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from IPython.display import HTML, display
#Код для аккуратного вывода длинных строк
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [4]:
import numpy as np
import torch

from transformers import BertTokenizer, DistilBertForQuestionAnswering, DistilBertTokenizerFast
import wikipedia as wiki

Проверим работу Википедии

In [5]:
wiki.set_lang('ru')

In [6]:
question = 'Пушкин'

results = wiki.search(question)
print("Резултаты поиска по запросу:\n")
print(results)

page = wiki.page(results[0])
text = page.content

Резултаты поиска по запросу:

['Пушкин, Александр Сергеевич', 'Пушкин (город)', 'Последняя дуэль и смерть Александра Пушкина', 'Пушкин, Сергей Львович', 'Гончарова, Наталья Николаевна', 'Потомки Пушкина', 'Пушкиния', 'Пушкин, Александр (значения)', 'Пушкины', 'Памятник А. С. Пушкину (Пушкин)']


In [7]:
print(text[:300])

Алекса́ндр Серге́евич Пу́шкин (26 мая [6 июня] 1799, Москва — 29 января [10 февраля] 1837, Санкт-Петербург) — русский поэт, драматург и прозаик, заложивший основы русского реалистического направления, литературный критик и теоретик литературы, историк, публицист, журналист; один из самых авторитетны


# Загрузка модели

In [8]:
# Подключим гугл-диск с сохраненной ранее обученной моделью
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [10]:
# Если нужно загрузить сохраненную на гугл диск модель, то можно воспользоваться следующим кодом
# Или можно загрузить модель с https://huggingface.co/models
PRE_TRAINED_MODEL_NAME = 'drive/MyDrive/Colab Notebooks/test-squad-trained'
tokenizer = DistilBertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME, local_files_only=True)

In [11]:
model=DistilBertForQuestionAnswering.from_pretrained(PRE_TRAINED_MODEL_NAME, local_files_only=True).to(device).eval()

In [12]:
def get_answer_to_question(question, Bert_model, n_search_pages=1, n_answers_per_page=10, max_answer_length = 30):
  # Находим все тексты по запросу из википедии
  results = wiki.search(question)
  texts=[]
  questions=[question]*n_search_pages
  print("По запросу найдены следующие страницы")
  for ind, result in enumerate(results[:n_search_pages]):
    page = wiki.page(result)
    print(page)
    texts.append(page.content)
    
  # Токенезируем все тексты
  tokenized_examples = tokenizer(
        questions,
        texts,
        truncation="only_second",
        stride=100,
        max_length=400,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",)
  
  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
  tokenized_examples["text_id"] = []
  tokenized_examples["offset index"] = []
  for i in range(len(tokenized_examples["input_ids"])):
    sequence_ids = tokenized_examples.sequence_ids(i)
    context_index = 1

    sample_index = sample_mapping[i]
    tokenized_examples["text_id"].append(sample_index)

    # Заполняем offset_mapping. Если токен не попадает в последовательность контекста, то запоминаем None
    tokenized_examples["offset_mapping"].append([
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])])
    
  with torch.no_grad():
    output= Bert_model(torch.LongTensor(tokenized_examples['input_ids']).to(device),
                       attention_mask=torch.LongTensor(tokenized_examples['attention_mask']).to(device))
  predictions = postprocess_qa_predictions(texts, tokenized_examples, output, 
                                           n_best_size = n_answers_per_page, max_answer_length = max_answer_length)
  return predictions

In [17]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    # Генерация финального предсказания ответа. Будем перебирать n_best_size ответов и выберем тот, у которого самый большой score
    all_start_logits, all_end_logits = raw_predictions['start_logits'].cpu().numpy(), raw_predictions['end_logits'].cpu().numpy()
    # Соответствие между примером из датасета и последовательностью токенов.
    features_per_example = {}
    for i, text_ind in enumerate(features['text_id']):
      if text_ind in features_per_example:
        features_per_example[text_ind].append(i)
      else:
        features_per_example[text_ind]=[i]

    valid_answers = []

    for example_index, example in enumerate(examples):
        # Индекс последовательности токенов, которая соответствует данному примеру.
        feature_indices = features_per_example[example_index]
        # Возьмем все последовательности токенов, соответствующие данному примеру.
        for feature_index in feature_indices:
            # Предсказание модели вероятности логитов.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # offset_mapping для восстановления ответа в изначальном контексте
            offset_mapping = features["offset_mapping"][feature_index]

            # Рассмотрим все варианты ответов для первых n_best_size возможных комбинаций начального и конечного логита 
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Выкинем невозможные ответы - это те ответы, у которых индексы токенов вылетели за границы или
                    # индексы не попали в область контекста
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[start_index])==0
                        or len(offset_mapping[end_index])==0
                    ):
                        continue
                    # Не рассматриваем ответы с длиной < 0, либо > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": example[start_char: end_char]
                        }
                    )
        
      # Выбираем лучший ответ
    if len(valid_answers) > 0:
      best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
    else:
      # Если у нас нет ненулевых ответов, то выдаем нулевой ответ
      best_answer = {"text": "Не получилось найти ответ, пожалуйста, переформулируйте вопрос", "score": 0.0}

    return best_answer

Протестируем работу системы на нескольких вопросах

In [18]:
question = 'Последний император России?'
get_answer_to_question(question, model, n_search_pages=2, max_answer_length=100)

По запросу найдены следующие страницы
<WikipediaPage 'Николай II'>
<WikipediaPage 'Последний император'>


{'score': 10.712574, 'text': 'Николая II'}

In [19]:
question = 'Когда родился Пушкин?'
get_answer_to_question(question, model, n_search_pages=1, max_answer_length=100)

По запросу найдены следующие страницы
<WikipediaPage 'Пушкин, Александр Сергеевич'>


{'score': 11.1438055, 'text': '26 мая (6 июня) 1799 г.'}

In [20]:
question = 'От чего умер Пушкин?'
get_answer_to_question(question, model, n_search_pages=1, max_answer_length=100)

По запросу найдены следующие страницы
<WikipediaPage 'Последняя дуэль и смерть Александра Пушкина'>


{'score': 12.046791, 'text': 'от перитонита'}

In [23]:
question = 'Какие страны входили в Антанту?'
get_answer_to_question(question, model, n_search_pages=2, max_answer_length=200)

По запросу найдены следующие страницы
<WikipediaPage 'Иностранная военная интервенция в России'>
<WikipediaPage 'Первая мировая война'>


{'score': 13.81327,
 'text': 'Российская империя, Британская империя, Французская республика и союзники'}

In [24]:
question = 'Что такое инфляция?'
get_answer_to_question(question, model, n_search_pages=2, max_answer_length=200)

По запросу найдены следующие страницы
<WikipediaPage 'Инфляция'>
<WikipediaPage 'Галопирующая инфляция'>


{'score': 8.918312, 'text': 'повышение общего уровня цен на товары и услуги'}